In [2]:
from __future__ import absolute_import
from __future__ import division
from __future__ import print_function
from __future__ import unicode__literals
from rasa_core import utils
from rasa_core.agent import Agent
from rasa_core.policies.keras_policy import KerasPolicy
from rasa_core.policies.memoization import MemoizationPolicy
from rasa_core.policies.sklearn_policy import SklearnPolicy

SyntaxError: from __future__ imports must occur at the beginning of the file (cell_name, line 4)

In [3]:
from rasa_nlu.training_data import load_data

In [4]:
from rasa_nlu.model import Trainer

In [31]:
from rasa_nlu import config


In [32]:
from rasa_nlu.model import Interpreter

In [33]:
def train_horoscopebot(data_json,config_file,model_dir):
    training_data = load_data(data_json)
    trainer = Trainer(config.load(config_file))
    trainer.train(training_data)
    model_directory = trainer.persist(model_dir , fixed_model_name='horoscopebot')
    

In [34]:
def predict_intent(text):
    interpreter = Interpreter.load('./models/nlu/default/horoscopebot')
    print(interpreter.parse(text))

In [35]:
train_horoscopebot('./data/data.json', 'config.json', './models/nlu')

Epochs: 100%|████████████████████████████████████████████████| 300/300 [00:02<00:00, 115.97it/s, loss=0.084, acc=1.000]


In [36]:
predict_intent("I am looking for my horoscope for today. I am wondering if you can tell me that.")

INFO:tensorflow:Restoring parameters from ./models/nlu/default/horoscopebot\component_5_EmbeddingIntentClassifier.ckpt
{'intent': {'name': 'get_horoscope', 'confidence': 0.9651930332183838}, 'entities': [{'start': 20, 'end': 29, 'value': 'horoscope', 'entity': 'horoscope_today', 'confidence': 0.6393314033970037, 'extractor': 'CRFEntityExtractor'}], 'intent_ranking': [{'name': 'get_horoscope', 'confidence': 0.9651930332183838}, {'name': 'subscription', 'confidence': 0.0}, {'name': 'greeting', 'confidence': 0.0}, {'name': 'dob_intent', 'confidence': 0.0}], 'text': 'I am looking for my horoscope for today. I am wondering if you can tell me that.'}


Wow! Isn’t it magical? Our model has predicted this text with a confidence of 96
percent. You can see in the provided ipynb file that our model does well in predicting
other intents as well. This is the power of tensorflow and ML overall. 

In [11]:
predict_intent("I want the subscription")

INFO:tensorflow:Restoring parameters from ./models/nlu/default/horoscopebot\component_5_EmbeddingIntentClassifier.ckpt
{'intent': {'name': 'subscription', 'confidence': 0.9706328511238098}, 'entities': [{'start': 11, 'end': 23, 'value': 'subscription', 'entity': 'regular_updates', 'confidence': 0.9103532492749712, 'extractor': 'CRFEntityExtractor'}], 'intent_ranking': [{'name': 'subscription', 'confidence': 0.9706328511238098}, {'name': 'get_horoscope', 'confidence': 0.0}, {'name': 'greeting', 'confidence': 0.0}], 'text': 'I want the subscription'}


In [10]:
predict_intent("Hi")

INFO:tensorflow:Restoring parameters from ./models/nlu/default/horoscopebot\intent_classifier_tensorflow_embedding.ckpt
{'intent': {'name': 'greeting', 'confidence': 0.9509541392326355}, 'entities': [], 'intent_ranking': [{'name': 'greeting', 'confidence': 0.9509541392326355}, {'name': 'subscription', 'confidence': 0.1056637093424797}, {'name': 'get_horoscope', 'confidence': 0.0}], 'text': 'Hi'}


In [11]:
predict_intent("What a great day hi!")

INFO:tensorflow:Restoring parameters from ./models/nlu/default/horoscopebot\intent_classifier_tensorflow_embedding.ckpt
{'intent': {'name': 'greeting', 'confidence': 0.9634082317352295}, 'entities': [], 'intent_ranking': [{'name': 'greeting', 'confidence': 0.9634082317352295}, {'name': 'subscription', 'confidence': 0.0}, {'name': 'get_horoscope', 'confidence': 0.0}], 'text': 'What a great day hi!'}


In [38]:
predict_intent("My birthday is on 22-09")

INFO:tensorflow:Restoring parameters from ./models/nlu/default/horoscopebot\component_5_EmbeddingIntentClassifier.ckpt
{'intent': {'name': 'dob_intent', 'confidence': 0.9476463794708252}, 'entities': [], 'intent_ranking': [{'name': 'dob_intent', 'confidence': 0.9476463794708252}, {'name': 'get_horoscope', 'confidence': 0.11561068892478943}, {'name': 'greeting', 'confidence': 0.11244606971740723}, {'name': 'subscription', 'confidence': 0.0}], 'text': 'My birthday is on 22-09'}


In [37]:
from rasa_core.actions import Action
from rasa_core.events import SlotSet


In [39]:
class GetTodaysHoroscope(Action):
    def name(self):
        return "get_todays_horoscope"
    def run(self,dispatcher,tracker,domain):
        #type: (Dispatcher,DialogStateTracker,Domain) ->List[Event]
        user_horoscope_sign = tracker.get_slot('horoscope_sign')
        #logic to get the horoscope
        return [SlotSet('horoscope_sign',user_horoscope_sign)]        

In [40]:
from __future__ import absolute_import
from __future__ import division
from __future__ import print_function
from __future__ import unicode_literals

import requests
from rasa_core_sdk import Action
from rasa_core_sdk.events import SlotSet
class GetTodayHoroscope(Action):
    def name(self):
        return "get_todays_horoscope"
    def run(self, dispatcher,tracker,domain):
        #type : (Dispatcher, DialogueStateTracker,Domain) -. List[Event]
        user_horoscope_sign = tracker.get_slot('horoscope_sign')
        base_url = "http://horoscope-api.herokuapp.com/horoscope/today/{day}/{sign}"
        url = base_url.format(**{'day':"today",'sign':user_horoscope_sign}) 
        #http://horoscope-api.herokuapp.com/horoscope/today/capricorn
        res = requests.get(url)
        todays_horoscope = res.json()['horoscope']
        response = "Your today's horoscope:\n{}".format(todays_horoscope)
        dispatcher.utter_message(response)
        return [SlotSet('horoscope_sign',user_horoscope_sign)]
        

Slots are the ones who make the bot responsible for working like a human. Slots are like
a storage space where the information given by the user can be stored or a pre-fetched
information from a database or an API can also be used.

dispatcher: the dispatcher is used to send messages back to our
users. We can use dipatcher.utter_message() to achieve the same.

• tracker: the state tracker for the current user. We can access slot
values using tracker.get_slot(slot_name), and to get the most
recent user message we can use tracker.latest_message.text.

Templates in Rasa are used for utterances. An utterance template contains a set
of preset text to be sent to the user when some action is triggered. By having the same
name of the action as the utterance or by an action with custom code, we can send our
formatted message in templates to the user.

As per the documentation of Rasa, “The Domain defines the universe in which your
bot operates.

# Writing Custom Actions of the Chatbot

In [41]:
from __future__ import absolute_import
from __future__ import division
from __future__ import print_function
from __future__ import unicode_literals

In [42]:
import requests
from rasa_core_sdk import Action
from rasa_core_sdk.events import SlotSet

In [43]:
class GetTodaysHoroscope(Action):
    def name(self):
        return "get_todays_horoscope"
    def run(self,dispatcher, tracker, domain):
        

SyntaxError: unexpected EOF while parsing (<ipython-input-43-0f079226be7c>, line 5)

In [44]:
class SubscribeUser(Action):
    def name(self):
        return "subscribe_user"
    def run(self, dispatcher , tracker , domain):
        subscribe = tracker.get_slot('subscribe')
        if subscribe == 'True':
            response = "You're successfully subscribed"
        if subscribe == "False":
            response = "You're successfully unsubscribed"
        dispatcher.utter_message(response)
        return [SlotSet("subscribe",subscribe)]
    
        

The next step is the data. Rasa’s dialog management model is trained on actual
conversations that users and the chatbot do. The important point here is that those
conversations have to be converted into a story format

A story is nothing but an actual conversation between a user and a chatbot where
user inputs are converted into intents and entities while responses being returned
from the chatbot are treated as actions that the chatbot is supposed to trigger when it’s
required

In [45]:
from __future__ import absolute_import
from __future__ import division
from __future__ import print_function
from __future__ import unicode__literals
from rasa_core import utils
from rasa_core.agent import Agent
from rasa_core.policies.keras_policy import KerasPolicy
from rasa_core.policies.memoization import MemoizationPolicy
from rasa_core.policies.sklearn_policy import SklearnPolicy

SyntaxError: from __future__ imports must occur at the beginning of the file (cell_name, line 4)